In [1]:
import pandas as pd
import icartt
import os
import warnings
import re
from datetime import datetime
import csv
from datetime import datetime, timedelta
from netCDF4 import Dataset
import numpy as np
from scipy import stats
import glob

In [2]:
def time_average(df, COL_REF, window_size=None, time_step=None):
   """
   Complete time averaging function that averages ALL data points in the dataset,
   not just those around existing COL_REF values.
   
   Parameters:
   df: pandas DataFrame with columns Date, UTC, COL_REF, and other data columns
   COL_REF: string, name of the reference column 
   window_size: int or None, half-window size for averaging around each time point.
                If None, automatically detected from COL_REF spacing or data spacing.
   time_step: int or None, time step between averaging points.
              If None, automatically detected from COL_REF spacing or data spacing.
   
   Returns:
   pandas DataFrame with averaged values at regular time intervals
   """
   import pandas as pd
   import numpy as np
   import time
   
   print("Starting complete time averaging...")
   start_time = time.time()
   
   # Make a copy to avoid modifying original
   df_work = df.copy()
   
   # Store original column order
   original_column_order = list(df.columns)
   
   # Step 1: Determine window_size and time_step from existing COL_REF data
   ref_mask = df_work[COL_REF].notna() & (df_work[COL_REF] != '')
   ref_points = df_work[ref_mask].copy()
   
   if len(ref_points) >= 2:
       ref_points_sorted = ref_points.sort_values(['Date', 'UTC'])
       first_utc = ref_points_sorted.iloc[0]['UTC']
       second_utc = ref_points_sorted.iloc[1]['UTC']
       detected_spacing = abs(second_utc - first_utc)
       
       if window_size is None:
           window_size = max(1, detected_spacing // 2)
       if time_step is None:
           time_step = detected_spacing
           
       print(f"Detected from COL_REF - Spacing: {detected_spacing}, Window: {window_size}, Time step: {time_step}")
   else:
       # Fall back to detecting from overall data spacing
       print("Insufficient COL_REF data, detecting from overall data...")
       df_sorted_sample = df_work.sort_values(['Date', 'UTC']).head(1000)  # Sample for speed
       utc_diffs = df_sorted_sample['UTC'].diff().dropna()
       median_spacing = utc_diffs.median()
       
       if window_size is None:
           window_size = max(1, median_spacing // 2)
       if time_step is None:
           time_step = median_spacing
           
       print(f"Detected from data - Median spacing: {median_spacing}, Window: {window_size}, Time step: {time_step}")
   
   # Step 2: Sort data once by Date and UTC for efficient processing
   print("Sorting data...")
   df_sorted = df_work.sort_values(['Date', 'UTC']).reset_index(drop=True)
   sort_time = time.time()
   print(f"Sorting completed in {sort_time - start_time:.1f}s")
   
   # Get column information
   key_columns = ['Date', 'UTC', COL_REF]
   numeric_columns = []
   non_numeric_columns = []

   for col in df_sorted.columns:
       if col not in key_columns:
           if col in ['Organization', 'Campaign']:
               non_numeric_columns.append(col)
           else:
               numeric_columns.append(col)
   
   # Convert numeric data columns to numeric
   print("Converting columns to numeric...")
   numeric_data = {}
   for col in numeric_columns:
       numeric_data[col] = pd.to_numeric(df_sorted[col], errors='coerce')
   
   # Step 3: Create complete time grid for each date
   unique_dates = df_sorted['Date'].unique()
   all_results = []
   
   print(f"Processing {len(unique_dates)} unique dates...")
   
   for date_idx, date in enumerate(unique_dates):
       if date_idx % max(1, len(unique_dates) // 10) == 0:
           elapsed = time.time() - start_time
           print(f"Processing date {date_idx + 1}/{len(unique_dates)} ({(date_idx/len(unique_dates)*100):.1f}%) - Elapsed: {elapsed:.1f}s")
       
       # Get all data for this date
       date_mask = df_sorted['Date'] == date
       date_data = df_sorted[date_mask].copy()
       
       if len(date_data) == 0:
           continue
       
       # Create time grid aligned to COL_REF for this date
       min_utc = date_data['UTC'].min()
       max_utc = date_data['UTC'].max()
       
       # Find COL_REF points for this date to determine alignment
       date_ref_points = ref_points[ref_points['Date'] == date] if len(ref_points) > 0 else pd.DataFrame()
       
       if len(date_ref_points) > 0:
           # Align grid to COL_REF spacing but extend to cover full range
           first_ref_utc = date_ref_points['UTC'].min()
           
           # Calculate how many steps to go back to cover min_utc
           steps_back = int(np.ceil((first_ref_utc - min_utc) / time_step))
           grid_start = first_ref_utc - (steps_back * time_step)
           
           # Create grid covering full range with COL_REF alignment
           time_points = np.arange(grid_start, max_utc + time_step, time_step)
       else:
           # No COL_REF for this date, use regular grid
           time_points = np.arange(min_utc, max_utc + time_step, time_step)
       
       date_data_indices = np.where(date_mask)[0]
       date_utc_values = date_data['UTC'].values
       
       # Step 4: For each time point, create averaged data
       for time_point in time_points:
           window_start = time_point - window_size
           window_end = time_point + window_size
           
           # Use searchsorted for fast boundary finding
           start_idx = np.searchsorted(date_utc_values, window_start, side='left')
           end_idx = np.searchsorted(date_utc_values, window_end, side='right')
           
           if start_idx >= end_idx:
               continue
               
           # Get the actual DataFrame indices for this window
           window_indices = date_data_indices[start_idx:end_idx]
           
           # Step 5: Create averaged row
           averaged_row = {
               'Date': date,
               'UTC': time_point
           }
           
           # Handle COL_REF specially - only include if there's actual data
           col_ref_values = df_sorted.loc[window_indices, COL_REF]
           col_ref_valid = col_ref_values.dropna()
           col_ref_valid = col_ref_valid[col_ref_valid != '']
           
           if len(col_ref_valid) > 0:
               # If COL_REF data exists in window, average it
               averaged_row[COL_REF] = pd.to_numeric(col_ref_valid, errors='coerce').mean()
           else:
               # If no COL_REF data, leave as NaN
               averaged_row[COL_REF] = np.nan
           
           # Average numeric data columns
           for col in numeric_columns:
               col_values = numeric_data[col].iloc[window_indices]
               valid_values = col_values.dropna()
               if len(valid_values) > 0:
                   averaged_row[col] = valid_values.mean()
               else:
                   averaged_row[col] = np.nan
           
           # Handle non-numeric columns (Organization, Campaign)
           for col in non_numeric_columns:
               col_values = df_sorted.loc[window_indices, col]
               non_null_values = col_values.dropna()
               if len(non_null_values) > 0:
                   averaged_row[col] = non_null_values.iloc[0]
               else:
                   averaged_row[col] = np.nan
           
           all_results.append(averaged_row)
   
   # Step 6: Create result DataFrame
   print("Creating result DataFrame...")
   result_df = pd.DataFrame(all_results)
   
   # Preserve original column order
   if len(result_df) > 0:
       available_cols = [col for col in original_column_order if col in result_df.columns]
       result_df = result_df[available_cols]
   
   total_time = time.time() - start_time
   print(f"\nComplete processing finished!")
   print(f"Total time: {total_time:.1f}s ({total_time/60:.1f} minutes)")
   print(f"Generated {len(all_results)} averaged time points")
   print(f"Points with COL_REF data: {result_df[COL_REF].notna().sum()}")
   print(f"Points without COL_REF data: {result_df[COL_REF].isna().sum()}")
   
   return result_df

def read_time_average_list(csv_path):
   """
   Reads the time_average_list.csv and returns a dictionary mapping campaign names to column names.
   
   Parameters:
   csv_path (str): Path to the time_average_list.csv file
   
   Returns:
   dict: Dictionary with campaign names as keys and column names as values
   """
   import pandas as pd
   
   # Read the CSV file
   df = pd.read_csv(csv_path)
   
   # Create dictionary mapping from 'campaign' to 'column_name'
   time_average_dict = dict(zip(df['campaign'], df['column_name']))
   
   return time_average_dict

# Usage example:
# time_average_mapping = read_time_average_list('time_average_list.csv')
# print(time_average_mapping)
# 
# # To get column name for a specific campaign:
# col_ref = time_average_mapping.get('ACMEV', 'BC_Mass')  # Default to 'BC_Mass' if not found

def standardize_campaign_dataframes(avg_path):
    """
    Read all campaign_avg.csv files and standardize their column structure.
    
    Parameters:
    avg_path: string, path to directory containing campaign_avg.csv files
    
    Returns:
    dict: Dictionary with campaign names as keys and standardized DataFrames as values
    """
    
    # Define the required columns in order
    required_columns_base = [
        'Organization',
        'Campaign', 
        'UTC',
        'Date',
        'Latitude',
        'Longitude',
        'Altitude',
        'Temperature',
        'Rel_humidity',
        'Pressure',
        'U',
        'V', 
        'W',
        'Supersaturation',
        'Number_Concentration',
        'CNgt3nm',
        'CNgt10nm',
        'BC_Mass',
        'LWC'
    ]
    
    # Scattering and absorption columns
    scattering_absorption_columns = [
        'Sc450_total',
        'Sc550_total', 
        'Sc700_total',
        'Abs470_total',
        'Abs532_total',
        'Abs660_total'
    ]
    
    # Find all campaign_avg.csv files
    pattern = os.path.join(avg_path, "*_avg.csv")
    avg_files = glob.glob(pattern)
    
    if not avg_files:
        print(f"No *_avg.csv files found in {avg_path}")
        return {}
    
    print(f"Found {len(avg_files)} campaign average files")
    
    standardized_dfs = {}
    
    for file_path in avg_files:
        # Extract campaign name from filename
        filename = os.path.basename(file_path)
        campaign_name = filename.replace('_avg.csv', '')
        
        print(f"\nProcessing: {campaign_name}")
        
        try:
            # Read the CSV file
            df = pd.read_csv(file_path)
            print(f"  Original shape: {df.shape}")
            print(f"  Original columns: {len(df.columns)}")
            
            # Get existing bin and cbin columns
            existing_bin_cols = [col for col in df.columns if col.startswith('bin') and col != 'bin']
            existing_cbin_cols = [col for col in df.columns if col.startswith('cbin')]
            
            print(f"  Found {len(existing_bin_cols)} bin columns, {len(existing_cbin_cols)} cbin columns")
            
            # Determine bin and cbin columns to use
            # For bin columns (minimum 15)
            if existing_bin_cols:
                # Sort existing bin columns naturally (bin1, bin2, ..., bin10, bin11, etc.)
                def natural_sort_key(col):
                    import re
                    numbers = re.findall(r'\d+', col)
                    return int(numbers[0]) if numbers else 0
                
                existing_bin_cols.sort(key=natural_sort_key)
                max_bin_num = max([int(''.join(filter(str.isdigit, col))) for col in existing_bin_cols if any(c.isdigit() for c in col)])
                final_bin_count = max(15, max_bin_num)
            else:
                final_bin_count = 15
            
            # For cbin columns (minimum 20)  
            if existing_cbin_cols:
                existing_cbin_cols.sort(key=natural_sort_key)
                max_cbin_num = max([int(''.join(filter(str.isdigit, col))) for col in existing_cbin_cols if any(c.isdigit() for c in col)])
                final_cbin_count = max(20, max_cbin_num)
            else:
                final_cbin_count = 20
            
            # Generate bin and cbin column lists
            bin_columns = [f'bin{i}' for i in range(1, final_bin_count + 1)]
            cbin_columns = [f'cbin{i}' for i in range(1, final_cbin_count + 1)]
            
            print(f"  Will use {len(bin_columns)} bin columns (bin1-bin{final_bin_count})")
            print(f"  Will use {len(cbin_columns)} cbin columns (cbin1-cbin{final_cbin_count})")
            
            # Create the complete required column list
            required_columns = (required_columns_base + 
                              cbin_columns + 
                              bin_columns + 
                              scattering_absorption_columns)
            
            # Create a new DataFrame with all required columns
            standardized_df = pd.DataFrame(columns=required_columns)
            
            # Copy existing data for columns that exist
            for col in required_columns:
                if col in df.columns:
                    standardized_df[col] = df[col]
                else:
                    # Add missing column with NaN values
                    standardized_df[col] = np.nan
            
            # Ensure the DataFrame has the same number of rows as original
            if len(df) > 0:
                standardized_df = standardized_df.reindex(range(len(df)))
                
                # Re-copy the data to ensure proper length
                for col in required_columns:
                    if col in df.columns:
                        standardized_df[col] = df[col].values
            
            print(f"  Standardized shape: {standardized_df.shape}")
            print(f"  Added {len(required_columns) - len(df.columns)} missing columns")
            
            # Store the standardized DataFrame
            standardized_dfs[campaign_name] = standardized_df
            
        except Exception as e:
            print(f"  ERROR processing {campaign_name}: {str(e)}")
            continue
    
    print(f"\nSuccessfully standardized {len(standardized_dfs)} campaign datasets")
    return standardized_dfs

def save_standardized_dataframes(standardized_dfs, avg_path):
    """
    Save the standardized DataFrames back to the original CSV files, replacing them.
    
    Parameters:
    standardized_dfs: dict, Dictionary of standardized DataFrames
    avg_path: string, Directory containing the original campaign_avg.csv files
    """
    
    print(f"\nReplacing original files in: {avg_path}")
    
    for campaign_name, df in standardized_dfs.items():
        output_file = os.path.join(avg_path, f"{campaign_name}_avg.csv")
        df.to_csv(output_file, index=False)
        print(f"  Replaced: {campaign_name}_avg.csv ({df.shape[0]} rows, {df.shape[1]} columns)")
    
    print(f"All {len(standardized_dfs)} files have been replaced with standardized versions!")


In [4]:
# Define paths
raw_path = rf"C:\Users\haika\Desktop\May_Research\may_datasets\renamed_data"
avg_path = rf"C:\Users\haika\Desktop\May_Research\may_datasets\time_averaged_data"
time_average_list_path = rf"C:\Users\haika\Desktop\May_Research\may_datasets\time_average_list\time_average_list.csv"

# Read the time averaging mapping
time_average_mapping = read_time_average_list(time_average_list_path)
print(f"Loaded time averaging mapping for {len(time_average_mapping)} campaigns")

# Create output directory if it doesn't exist
os.makedirs(avg_path, exist_ok=True)

# Process each campaign in the mapping
for campaign_name, col_ref in time_average_mapping.items():
   print(f"\n{'='*50}")
   print(f"Processing campaign: {campaign_name}")
   print(f"Using reference column: {col_ref}")
   print(f"{'='*50}")
   
   # Define file paths
   input_path = os.path.join(raw_path, f"{campaign_name}_renamed.csv")
   output_path = os.path.join(avg_path, f"{campaign_name}_avg.csv")
   
   # Check if input file exists
   if not os.path.exists(input_path):
       print(f"WARNING: Input file not found: {input_path}")
       print("Skipping this campaign...")
       continue
   
   try:
       # Read the renamed campaign data
       print(f"Reading: {input_path}")
       df = pd.read_csv(input_path)
       print(f"Loaded {len(df)} rows with {len(df.columns)} columns")
       
       # Check if the reference column exists
       if col_ref not in df.columns:
           print(f"WARNING: Reference column '{col_ref}' not found in {campaign_name}")
           print(f"Available columns: {list(df.columns)}")
           print("Skipping this campaign...")
           continue
       
       # Perform time averaging
       print(f"Starting time averaging with reference column: {col_ref}")
       df_avg = time_average(df, col_ref)
       
       # Save the averaged data
       print(f"Saving averaged data to: {output_path}")
       df_avg.to_csv(output_path, index=False)
       print(f"Successfully saved {len(df_avg)} averaged time points")
       
   except Exception as e:
       print(f"ERROR processing {campaign_name}: {str(e)}")
       print("Skipping this campaign...")
       continue

print(f"\n{'='*50}")
print("Time averaging processing completed!")
print(f"Check output directory: {avg_path}")
print(f"{'='*50}")

Loaded time averaging mapping for 1 campaigns

Processing campaign: BBOP
Using reference column: BC_Mass
Reading: C:\Users\haika\Desktop\May_Research\may_datasets\renamed_data\BBOP_renamed.csv
Loaded 454849 rows with 145 columns
Starting time averaging with reference column: BC_Mass
Starting complete time averaging...
Detected from COL_REF - Spacing: 10.0, Window: 5.0, Time step: 10.0
Sorting data...
Sorting completed in 0.4s
Converting columns to numeric...
Processing 35 unique dates...
Processing date 1/35 (0.0%) - Elapsed: 0.4s
Processing date 4/35 (8.6%) - Elapsed: 79.5s
Processing date 7/35 (17.1%) - Elapsed: 182.4s
Processing date 10/35 (25.7%) - Elapsed: 241.9s
Processing date 13/35 (34.3%) - Elapsed: 370.1s
Processing date 16/35 (42.9%) - Elapsed: 490.4s
Processing date 19/35 (51.4%) - Elapsed: 587.3s
Processing date 22/35 (60.0%) - Elapsed: 640.8s
Processing date 25/35 (68.6%) - Elapsed: 716.6s
Processing date 28/35 (77.1%) - Elapsed: 818.7s
Processing date 31/35 (85.7%) - Ela

In [5]:
# Define path
avg_path = rf"C:\Users\haika\Desktop\May_Research\may_datasets\time_averaged_data"

# Standardize all campaign DataFrames
print("Starting standardization process...")
standardized_campaigns = standardize_campaign_dataframes(avg_path)

# Display summary information
if standardized_campaigns:
    print(f"\nStandardization Summary:")
    print("-" * 50)
    for campaign_name, df in standardized_campaigns.items():
        non_empty_cols = df.count().sum()
        total_cols = len(df.columns)
        print(f"{campaign_name}: {df.shape[0]} rows, {total_cols} columns ({non_empty_cols} non-empty values)")
    
    # Replace original files with standardized versions
    save_standardized_dataframes(standardized_campaigns, avg_path)

Starting standardization process...
Found 9 campaign average files

Processing: ACE-ENA
  Original shape: (546787, 76)
  Original columns: 76
  Found 30 bin columns, 21 cbin columns
  Will use 30 bin columns (bin1-bin30)
  Will use 21 cbin columns (cbin1-cbin21)
  Standardized shape: (546787, 76)
  Added 0 missing columns

Processing: ACMEV
  Original shape: (55849, 142)
  Original columns: 142
  Found 87 bin columns, 30 cbin columns
  Will use 87 bin columns (bin1-bin87)
  Will use 30 cbin columns (cbin1-cbin30)
  Standardized shape: (55849, 142)
  Added 0 missing columns

Processing: BBOP
  Original shape: (64847, 145)
  Original columns: 145
  Found 91 bin columns, 30 cbin columns
  Will use 91 bin columns (bin1-bin91)
  Will use 30 cbin columns (cbin1-cbin30)
  Standardized shape: (64847, 146)
  Added 1 missing columns

Processing: CACTI
  Original shape: (38596, 145)
  Original columns: 145
  Found 99 bin columns, 21 cbin columns
  Will use 99 bin columns (bin1-bin99)
  Will use 2